In [1]:
%load_ext autoreload
%autoreload 1

In [ ]:
import os
from pathlib import Path
import tempfile

import gymnasium
import numpy as np

%aimport minetest

repo_root = Path(globals()['_dh'][0]).parent.parent
temp_dir = tempfile.mkdtemp(prefix="minetest_")
game_dir = os.path.join(os.environ["CONDA_PREFIX"], "share/minetest/games/devtest")
# install devtest game if needed
if not os.path.exists(game_dir):
    os.makedirs(os.path.dirname(game_dir), exist_ok=True)
    os.symlink(os.path.realpath(os.path.join(repo_root, "games/devtest")), game_dir)

# use minetest from repo if needed
minetest_executable = os.path.join(os.environ["CONDA_PREFIX"], "bin/minetest")
if not os.path.exists(minetest_executable):
    minetest_executable = os.path.join(repo_root, "bin/minetest")
print(f"Using minetest executable: {minetest_executable}")

env = minetest.MinetestEnv(
    executable=minetest_executable,
    artifact_dir=os.path.join(temp_dir, "artifacts"),
    game_dir=game_dir,
)
video_dir = os.path.join(temp_dir, "video")
empty_action = {"KEYS": np.ndarray(0), "MOUSE": [0, 0]}
with gymnasium.wrappers.RecordVideo(env, video_dir) as env:
    env.reset()
    for i in range(1000):
        if i % 10 == 0:
            action = env.action_space.sample()
        else:
            action = empty_action
        observation, reward, done, truncated, info = env.step(action)
        if done: break


## Note
If you see `Irrlicht: GL_INVALID_OPERATION` on the recording, this will not happen in a Release build.
We see that with SDL2 rendering but not X11, so it's probably an upstream bug.
Doesn't seem to break anything though.

In [ ]:
import io
import base64
from IPython.display import HTML

video = io.open(os.path.join(video_dir, os.listdir(video_dir)[-1]), "r+b").read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode("ascii")))